In [ ]:
%cd ..

In [ ]:
import pandas as pd
import re
import os

exp_path = "./yolov5/runs/detect/exp"
file_list = []
for file in os.listdir(f"{exp_path}/labels"):
    if file.endswith(".txt"):
        file_list.append(file)
df = []
for file in file_list:
    df.append({"spot": re.search(r'spot_(.*?)_date', file).group(1),
               "date": re.search(r'date_(.*?).txt', file).group(1),
               "file": file})
df = pd.DataFrame(df)
df["n_surfers_pred"] = df.file.apply(lambda x: len(open(f"{exp_path}/labels/"+x).read().splitlines()))
df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d_%H-%M')
df.head()

In [10]:
with open(f'{exp_path}/labels/spot_Lacanau_date_2022-07-11_12-31.txt') as f:
    lines = f.read().splitlines()
    print(lines)

['0 0.630859 0.661806 0.00703125 0.0125', '0 0.154687 0.831944 0.0203125 0.0388889', '0 0.507422 0.777778 0.0101562 0.0277778', '0 0.618359 0.645833 0.0257812 0.0166667', '0 0.630859 0.668056 0.0101562 0.0222222', '0 0.629297 0.680556 0.0101562 0.0194444', '0 0.621484 0.640278 0.0117188 0.0111111', '0 0.610156 0.648611 0.0203125 0.0138889', '0 0.724219 0.661806 0.009375 0.0208333', '0 0.0304688 0.828472 0.01875 0.0402778', '0 0.726562 0.640278 0.0140625 0.0138889', '0 0.748047 0.664583 0.0101562 0.0208333', '0 0.404687 0.674306 0.01875 0.0180556', '0 0.450781 0.654861 0.0203125 0.0291667', '0 0.487109 0.654167 0.0132813 0.0222222', '0 0.437891 0.794444 0.00703125 0.0166667']


In [11]:
import plotly.express as px

fig = px.line(df[df.date >= '2022-07-01'], x='date', y="n_surfers_pred", color="spot",
             width=1300, height=800)
fig.show()

In [5]:
labels_path = "data/splitted_images/annotated/labels/val"
file_list = []
for file in os.listdir(f"{labels_path}"):
    if file.endswith(".txt"):
        file_list.append(file)
print(len(file_list))

df2 = []
for file in file_list:
    df2.append({"spot": re.search(r'spots_(.*?)_2', file).group(1),
               "date": re.search(r'_(2.*?).txt', file).group(1),
               "file_label": file})
df2 = pd.DataFrame(df2)
df2["n_surfers_true"] = df2.file_label.apply(lambda x: len(open(f"{labels_path}/"+x).read().splitlines()))
df2['date'] = pd.to_datetime(df2.date,format='%Y-%m-%d_%H-%M')
df2

18


,spot,date,file_label,n_surfers_true
0,Anglet_GrandePlage,2022-03-25 10:30:00,spots_Anglet_GrandePlage_2022-03-25_10-30.txt,1
1,Biarritz,2022-03-14 14:41:00,spots_Biarritz_2022-03-14_14-41.txt,6
2,Biarritz,2022-03-18 14:30:00,spots_Biarritz_2022-03-18_14-30.txt,18
3,Biarritz,2022-03-20 10:45:00,spots_Biarritz_2022-03-20_10-45.txt,11
4,Biarritz,2022-03-21 08:30:00,spots_Biarritz_2022-03-21_08-30.txt,10
5,Biarritz,2022-03-21 16:00:00,spots_Biarritz_2022-03-21_16-00.txt,5
6,Biarritz,2022-03-24 11:15:00,spots_Biarritz_2022-03-24_11-15.txt,20
7,Biarritz,2022-03-26 18:30:00,spots_Biarritz_2022-03-26_18-30.txt,1
8,Biarritz,2022-03-27 09:15:00,spots_Biarritz_2022-03-27_09-15.txt,3
9,Biarritz,2022-03-29 18:45:00,spots_Biarritz_2022-03-29_18-45.txt,3


In [6]:
df2 = df.merge(df2, on=["spot", "date"])

In [7]:
fig = px.scatter(df2, x='n_surfers_true', y="n_surfers_pred", color="spot", custom_data=["file_label"],
             width=1300, height=800)
fig.update_traces(
    hovertemplate="<br>".join([
        "true: %{x}",
        "pred: %{y}",
        "file: %{customdata[0]}",
    ])
)
fig.show()